参考：
- https://en.wikipedia.org/wiki/Viterbi_algorithm
- https://www.zhihu.com/question/20136144

```
"Viterbi path" and "Viterbi algorithm" have become standard terms for the 
application of dynamic programming algorithms to maximization problems involving 
probabilities.
```

In [27]:
"""
医生通过观察病人的状态来判断病人的健康状况，输入是病人每一天的状态（正常、冷、眩晕），输出是每一天的身体状态标签（健康 or 发烧）
"""

# 病人的隐藏状态
states = ('Healthy', 'Fever')

# 病人起始的隐藏状态
start_p = {'Healthy': 0.6, 'Fever': 0.4}

# 病人隐藏状态的转移概率，例如昨天是 Healthy，今天也是 Healthy 的概率为 0.7
trans_p = {
   'Healthy' : {'Healthy': 0.7, 'Fever': 0.3},
   'Fever' : {'Healthy': 0.4, 'Fever': 0.6}
}

# 如果病人处于某个隐藏状态，表现出各种显示状态的概率
emit_p = {
   'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
   'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6}
}

# 医生观察到的病人每天的状态
obs = ('normal', 'cold', 'dizzy')

def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    
    # 计算病人的初始隐藏状态
    for st in states:
        V[0][st] = {
            "prob": start_p[st] * emit_p[st][obs[0]], 
            "prev": None
        }
    
    for t in range(1, len(obs)):
        V.append({})
        
        # 这里两层 for 循环的目的是为了根据上一步所有隐藏状态的概率计算出转移到当前步隐藏状态的所有概率
        # 第一层迭代是迭代当前状态
        for st in states:
            # 第二层迭代是迭代前一步的所有隐藏状态
            # 初始化前一步的隐藏状态，设为所有状态当中的第一个状态
            max_tr_prob = V[t-1][states[0]]["prob"] * trans_p[states[0]][st]
            prev_st_selected = states[0]  #
            
            # 前一步的隐藏状态，遍历剩下的状态
            for prev_st in states[1:]:
                tr_prob = V[t-1][prev_st]["prob"] * trans_p[prev_st][st]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st
        
            max_prob = max_tr_prob * emit_p[st][obs[t]]
            V[t][st] = {"prob": max_prob, "prev": prev_st_selected}
            
    for line in dptable(V):
        print(line)
    
    # 获得最后一步概率最高的状态
    max_prob = max(V[-1].items(), key=lambda x: x[1]["prob"])
    print('last step max prob:', max_prob)
    previous = max_prob[1]['prev']
    
    opt = [max_prob[0]]
    for t in range(len(V)-2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]
    
    print(f"The steps of states are: {opt} with highest probability: {max_prob[1]['prob']}")
        
            
def dptable(V):
    # Print a table of steps from dictionary
    yield " ".join(("%12d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%f" % v[state]["prob"]) for v in V)

viterbi(obs, states, start_p, trans_p, emit_p)

           0            1            2
Healthy: 0.30000 0.08400 0.00588
Fever: 0.04000 0.02700 0.01512
last step max prob: ('Fever', {'prob': 0.01512, 'prev': 'Healthy'})
The steps of states are: ['Healthy', 'Healthy', 'Fever'] with highest probability: 0.01512
